In [1]:
import sys
import io
import time
import requests
import datetime
import numpy as np
import pandas as pd

sys.path.append('../')
from src.util.functions import Functions

# Disable warning
pd.options.mode.chained_assignment = None 

level = 'regioni'

path_data = '../data'
path_csv = '{path_data}/csv'.format(path_data=path_data)

csv_out_name = '{path_csv}/covid19-{level}.csv'.format(path_csv=path_csv, level=level)
pop_regions_name = '{path_csv}/popolazione-regioni.csv'.format(path_csv=path_csv)
icu_regions_name = '{path_csv}/terapie-intensive.csv'.format(path_csv=path_csv)
regions_name = '{path_csv}/regioni.csv'.format(path_csv=path_csv)

In [2]:
url_cum = Functions.get_url_cum(level)
url_cum

'https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/dati-regioni/dpc-covid19-ita-regioni.csv'

In [3]:
df_raw_r = pd.read_csv(url_cum)
df_raw_r

,data,stato,codice_regione,denominazione_regione,lat,long,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,...,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,note
0,2020-02-24T18:00:00,ITA,13,Abruzzo,42.351222,13.398438,0,0,0,0,...,0,0,0,0,NaN,NaN,0,5,NaN,NaN
1,2020-02-24T18:00:00,ITA,17,Basilicata,40.639471,15.805148,0,0,0,0,...,0,0,0,0,NaN,NaN,0,0,NaN,NaN
2,2020-02-24T18:00:00,ITA,18,Calabria,38.905976,16.594402,0,0,0,0,...,0,0,0,0,NaN,NaN,0,1,NaN,NaN
3,2020-02-24T18:00:00,ITA,15,Campania,40.839566,14.250850,0,0,0,0,...,0,0,0,0,NaN,NaN,0,10,NaN,NaN
4,2020-02-24T18:00:00,ITA,8,Emilia-Romagna,44.494367,11.341721,10,2,12,6,...,0,18,0,0,NaN,NaN,18,148,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5686,2020-11-20T17:00:00,ITA,19,Sicilia,38.115697,13.362357,1537,242,1779,32977,...,1175,1634,14179,1098,31028.0,19005.0,50033,866503,596565.0,NaN
5687,2020-11-20T17:00:00,ITA,9,Toscana,43.769231,11.255889,1792,295,2087,51782,...,-255,2207,34894,2121,71442.0,19442.0,90884,1421783,897193.0,effettuati 58204 tamponi rapidi antigenici di...
5688,2020-11-20T17:00:00,ITA,10,Umbria,43.106758,12.388247,363,71,434,10934,...,-17,394,9279,314,5879.0,15082.0,20961,380989,211832.0,NaN
5689,2020-11-20T17:00:00,ITA,2,Valle d'Aosta,45.737503,7.320149,148,10,158,1922,...,45,155,3474,278,5186.0,646.0,5832,53347,30994.0,NaN


In [4]:
# Drop useless columns
cols_to_drop = ['stato', 'lat', 'long', 'note']
df_rd = df_raw_r.drop(cols_to_drop, axis=1)
df_rd

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati
0,2020-02-24T18:00:00,13,Abruzzo,0,0,0,0,0,0,0,0,0,NaN,NaN,0,5,NaN
1,2020-02-24T18:00:00,17,Basilicata,0,0,0,0,0,0,0,0,0,NaN,NaN,0,0,NaN
2,2020-02-24T18:00:00,18,Calabria,0,0,0,0,0,0,0,0,0,NaN,NaN,0,1,NaN
3,2020-02-24T18:00:00,15,Campania,0,0,0,0,0,0,0,0,0,NaN,NaN,0,10,NaN
4,2020-02-24T18:00:00,8,Emilia-Romagna,10,2,12,6,18,0,18,0,0,NaN,NaN,18,148,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5686,2020-11-20T17:00:00,19,Sicilia,1537,242,1779,32977,34756,1175,1634,14179,1098,31028.0,19005.0,50033,866503,596565.0
5687,2020-11-20T17:00:00,9,Toscana,1792,295,2087,51782,53869,-255,2207,34894,2121,71442.0,19442.0,90884,1421783,897193.0
5688,2020-11-20T17:00:00,10,Umbria,363,71,434,10934,11368,-17,394,9279,314,5879.0,15082.0,20961,380989,211832.0
5689,2020-11-20T17:00:00,2,Valle d'Aosta,148,10,158,1922,2080,45,155,3474,278,5186.0,646.0,5832,53347,30994.0


In [5]:
last_date_str = df_rd.iloc[-1].data.split('T')[0]
last_date = datetime.datetime.strptime(last_date_str, '%Y-%m-%d')
prev_date = Functions.get_previous_date(last_date)
prev_date_str = datetime.datetime.strftime(prev_date, '%Y-%m-%d')
last_date_str, prev_date_str

('2020-11-20', '2020-11-19')

In [6]:
# Current state S
df_raw_r_1 = df_rd[df_rd.data.str.startswith(last_date_str)].sort_values(by='codice_regione').reset_index(drop=True)
df_raw_r_1

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati
0,2020-11-20T17:00:00,1,Piemonte,5225,393,5618,70927,76545,-692,3861,62074,5419,62133.0,81905.0,144038,1386198,875229.0
1,2020-11-20T17:00:00,2,Valle d'Aosta,148,10,158,1922,2080,45,155,3474,278,5186.0,646.0,5832,53347,30994.0
2,2020-11-20T17:00:00,3,Lombardia,8304,930,9234,150407,159641,4061,9221,177832,20190,267227.0,90436.0,357663,3724757,2221955.0
3,2020-11-20T17:00:00,5,Veneto,2170,267,2437,66250,68687,2105,3468,44349,3123,29424.0,86735.0,116159,2640734,1016938.0
4,2020-11-20T17:00:00,6,Friuli Venezia Giulia,505,54,559,11616,12175,530,1018,10157,608,19425.0,3515.0,22940,651014,261532.0
5,2020-11-20T17:00:00,7,Liguria,1290,120,1410,14544,15954,-1073,761,28594,2207,32198.0,14557.0,46755,553862,275463.0
6,2020-11-20T17:00:00,8,Emilia-Romagna,2534,244,2778,60156,62934,1925,2533,32145,5265,64026.0,36318.0,100344,1957785,1011858.0
7,2020-11-20T17:00:00,9,Toscana,1792,295,2087,51782,53869,-255,2207,34894,2121,71442.0,19442.0,90884,1421783,897193.0
8,2020-11-20T17:00:00,10,Umbria,363,71,434,10934,11368,-17,394,9279,314,5879.0,15082.0,20961,380989,211832.0
9,2020-11-20T17:00:00,11,Marche,542,81,623,14354,14977,121,512,9475,1161,19698.0,5915.0,25613,384786,224325.0


In [7]:
# State S - 1
df_raw_r_0 = df_rd[df_rd.data.str.startswith(prev_date_str)].sort_values(by='codice_regione').reset_index(drop=True)
df_raw_r_0

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati
0,2020-11-19T17:00:00,1,Piemonte,5147,390,5537,71700,77237,2173,5349,57609,5331,58272.0,81905.0,140177,1364685,865428.0
1,2020-11-19T17:00:00,2,Valle d'Aosta,144,12,156,1879,2035,-58,91,3371,271,5042.0,635.0,5677,52636,30688.0
2,2020-11-19T17:00:00,3,Lombardia,8291,915,9206,146374,155580,3497,7453,172847,20015,261506.0,86936.0,348442,3682509,2208742.0
3,2020-11-19T17:00:00,5,Veneto,2121,279,2400,64182,66582,2276,3753,43052,3057,28879.0,83812.0,112691,2624768,1011752.0
4,2020-11-19T17:00:00,6,Friuli Venezia Giulia,485,48,533,11112,11645,891,1197,9697,580,18558.0,3364.0,21922,643492,259446.0
5,2020-11-19T17:00:00,7,Liguria,1319,119,1438,15589,17027,259,792,26789,2178,31765.0,14229.0,45994,548215,273332.0
6,2020-11-19T17:00:00,8,Emilia-Romagna,2489,244,2733,58276,61009,1690,2160,31587,5218,62529.0,35285.0,97814,1935478,1003409.0
7,2020-11-19T17:00:00,9,Toscana,1814,287,2101,52023,54124,14,1972,32480,2073,69971.0,18706.0,88677,1403341,888853.0
8,2020-11-19T17:00:00,10,Umbria,369,75,444,10941,11385,111,556,8889,293,5727.0,14840.0,20567,377286,210121.0
9,2020-11-19T17:00:00,11,Marche,526,78,604,14252,14856,103,667,9095,1150,19186.0,5915.0,25101,381419,222600.0


In [8]:
# New df regions state S with Trentino-Alto Adige
df_r_1 = Functions.replace_bt_with_taa(df_raw_r_1)
df_r_1.sort_values(by='codice_regione').reset_index(drop=True)

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati
0,2020-11-20T17:00:00,1,Piemonte,5225,393,5618,70927,76545,-692,3861,62074,5419,62133.0,81905.0,144038,1386198,875229.0
1,2020-11-20T17:00:00,2,Valle d'Aosta,148,10,158,1922,2080,45,155,3474,278,5186.0,646.0,5832,53347,30994.0
2,2020-11-20T17:00:00,3,Lombardia,8304,930,9234,150407,159641,4061,9221,177832,20190,267227.0,90436.0,357663,3724757,2221955.0
3,2020-11-20T17:00:00,4,Trentino-Alto Adige,885,76,961,13554,14515,325,970,18354,1009,27295.0,6583.0,33878,632736,268359.0
4,2020-11-20T17:00:00,5,Veneto,2170,267,2437,66250,68687,2105,3468,44349,3123,29424.0,86735.0,116159,2640734,1016938.0
5,2020-11-20T17:00:00,6,Friuli Venezia Giulia,505,54,559,11616,12175,530,1018,10157,608,19425.0,3515.0,22940,651014,261532.0
6,2020-11-20T17:00:00,7,Liguria,1290,120,1410,14544,15954,-1073,761,28594,2207,32198.0,14557.0,46755,553862,275463.0
7,2020-11-20T17:00:00,8,Emilia-Romagna,2534,244,2778,60156,62934,1925,2533,32145,5265,64026.0,36318.0,100344,1957785,1011858.0
8,2020-11-20T17:00:00,9,Toscana,1792,295,2087,51782,53869,-255,2207,34894,2121,71442.0,19442.0,90884,1421783,897193.0
9,2020-11-20T17:00:00,10,Umbria,363,71,434,10934,11368,-17,394,9279,314,5879.0,15082.0,20961,380989,211832.0


In [9]:
# New df regions state S - 1 with Trentino-Alto Adige
df_r_0 = Functions.replace_bt_with_taa(df_raw_r_0)
df_r_0.sort_values(by='codice_regione').reset_index()

,index,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati
0,0,2020-11-19T17:00:00,1,Piemonte,5147,390,5537,71700,77237,2173,5349,57609,5331,58272.0,81905.0,140177,1364685,865428.0
1,1,2020-11-19T17:00:00,2,Valle d'Aosta,144,12,156,1879,2035,-58,91,3371,271,5042.0,635.0,5677,52636,30688.0
2,2,2020-11-19T17:00:00,3,Lombardia,8291,915,9206,146374,155580,3497,7453,172847,20015,261506.0,86936.0,348442,3682509,2208742.0
3,19,2020-11-19T17:00:00,4,Trentino-Alto Adige,876,78,954,13236,14190,429,962,17724,994,26477.0,6431.0,32908,626406,266696.0
4,3,2020-11-19T17:00:00,5,Veneto,2121,279,2400,64182,66582,2276,3753,43052,3057,28879.0,83812.0,112691,2624768,1011752.0
5,4,2020-11-19T17:00:00,6,Friuli Venezia Giulia,485,48,533,11112,11645,891,1197,9697,580,18558.0,3364.0,21922,643492,259446.0
6,5,2020-11-19T17:00:00,7,Liguria,1319,119,1438,15589,17027,259,792,26789,2178,31765.0,14229.0,45994,548215,273332.0
7,6,2020-11-19T17:00:00,8,Emilia-Romagna,2489,244,2733,58276,61009,1690,2160,31587,5218,62529.0,35285.0,97814,1935478,1003409.0
8,7,2020-11-19T17:00:00,9,Toscana,1814,287,2101,52023,54124,14,1972,32480,2073,69971.0,18706.0,88677,1403341,888853.0
9,8,2020-11-19T17:00:00,10,Umbria,369,75,444,10941,11385,111,556,8889,293,5727.0,14840.0,20567,377286,210121.0


In [10]:
df_r = df_r_1.copy()
# Merge with population data
df_pop_r = pd.read_csv(pop_regions_name)
df_m_r_1 = df_r.merge(df_pop_r, on='denominazione_regione')
df_m_r_1.sort_values(by='codice_regione').reset_index(drop=True)

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,popolazione
0,2020-11-20T17:00:00,1,Piemonte,5225,393,5618,70927,76545,-692,3861,62074,5419,62133.0,81905.0,144038,1386198,875229.0,4341375
1,2020-11-20T17:00:00,2,Valle d'Aosta,148,10,158,1922,2080,45,155,3474,278,5186.0,646.0,5832,53347,30994.0,125501
2,2020-11-20T17:00:00,3,Lombardia,8304,930,9234,150407,159641,4061,9221,177832,20190,267227.0,90436.0,357663,3724757,2221955.0,10103969
3,2020-11-20T17:00:00,4,Trentino-Alto Adige,885,76,961,13554,14515,325,970,18354,1009,27295.0,6583.0,33878,632736,268359.0,1074819
4,2020-11-20T17:00:00,5,Veneto,2170,267,2437,66250,68687,2105,3468,44349,3123,29424.0,86735.0,116159,2640734,1016938.0,4907704
5,2020-11-20T17:00:00,6,Friuli Venezia Giulia,505,54,559,11616,12175,530,1018,10157,608,19425.0,3515.0,22940,651014,261532.0,1211357
6,2020-11-20T17:00:00,7,Liguria,1290,120,1410,14544,15954,-1073,761,28594,2207,32198.0,14557.0,46755,553862,275463.0,1543127
7,2020-11-20T17:00:00,8,Emilia-Romagna,2534,244,2778,60156,62934,1925,2533,32145,5265,64026.0,36318.0,100344,1957785,1011858.0,4467118
8,2020-11-20T17:00:00,9,Toscana,1792,295,2087,51782,53869,-255,2207,34894,2121,71442.0,19442.0,90884,1421783,897193.0,3722729
9,2020-11-20T17:00:00,10,Umbria,363,71,434,10934,11368,-17,394,9279,314,5879.0,15082.0,20961,380989,211832.0,880285


In [11]:
# Merge with icu data
df_icu_r = pd.read_csv(icu_regions_name)
df_m_r_2 = df_m_r_1.merge(df_icu_r, on='denominazione_regione')
df_m_r_2.sort_values(by='codice_regione').reset_index(drop=True)

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,popolazione,totale_posti_terapia_intensiva
0,2020-11-20T17:00:00,1,Piemonte,5225,393,5618,70927,76545,-692,3861,62074,5419,62133.0,81905.0,144038,1386198,875229.0,4341375,575
1,2020-11-20T17:00:00,2,Valle d'Aosta,148,10,158,1922,2080,45,155,3474,278,5186.0,646.0,5832,53347,30994.0,125501,20
2,2020-11-20T17:00:00,3,Lombardia,8304,930,9234,150407,159641,4061,9221,177832,20190,267227.0,90436.0,357663,3724757,2221955.0,10103969,1036
3,2020-11-20T17:00:00,4,Trentino-Alto Adige,885,76,961,13554,14515,325,970,18354,1009,27295.0,6583.0,33878,632736,268359.0,1074819,106
4,2020-11-20T17:00:00,5,Veneto,2170,267,2437,66250,68687,2105,3468,44349,3123,29424.0,86735.0,116159,2640734,1016938.0,4907704,1016
5,2020-11-20T17:00:00,6,Friuli Venezia Giulia,505,54,559,11616,12175,530,1018,10157,608,19425.0,3515.0,22940,651014,261532.0,1211357,180
6,2020-11-20T17:00:00,7,Liguria,1290,120,1410,14544,15954,-1073,761,28594,2207,32198.0,14557.0,46755,553862,275463.0,1543127,209
7,2020-11-20T17:00:00,8,Emilia-Romagna,2534,244,2778,60156,62934,1925,2533,32145,5265,64026.0,36318.0,100344,1957785,1011858.0,4467118,634
8,2020-11-20T17:00:00,9,Toscana,1792,295,2087,51782,53869,-255,2207,34894,2121,71442.0,19442.0,90884,1421783,897193.0,3722729,523
9,2020-11-20T17:00:00,10,Umbria,363,71,434,10934,11368,-17,394,9279,314,5879.0,15082.0,20961,380989,211832.0,880285,117


In [12]:
# Copy last merged df into df_r
df_r = df_m_r_2.copy()

In [13]:
# Add columns
df_r['variazione_ricoverati_con_sintomi'] = df_r_1['ricoverati_con_sintomi'] - df_r_0['ricoverati_con_sintomi']
df_r['variazione_terapia_intensiva'] = df_r_1['terapia_intensiva'] - df_r_0['terapia_intensiva']
df_r['variazione_totale_ospedalizzati'] = df_r_1['totale_ospedalizzati'] - df_r_0['totale_ospedalizzati']
df_r['posti_disponibili_terapia_intensiva'] = df_r['totale_posti_terapia_intensiva'] - df_r['terapia_intensiva']
df_r['saturazione_terapia_intensiva'] = ((df_r_1['terapia_intensiva'] / df_r['totale_posti_terapia_intensiva']) * 100).round(2)
df_r['variazione_isolamento_domiciliare'] = df_r_1['isolamento_domiciliare'] - df_r_0['isolamento_domiciliare']
df_r['variazione_nuovi_positivi'] = df_r_1['nuovi_positivi'] - df_r_0['nuovi_positivi']
df_r['nuovi_tamponi'] = df_r_1['tamponi'] - df_r_0['tamponi']
df_r['nuovi_casi_testati'] = df_r_1['casi_testati'] - df_r_0['casi_testati']
df_r['nuovi_deceduti'] = df_r_1['deceduti'] - df_r_0['deceduti']
df_r['nuovi_dimessi_guariti'] = df_r_1['dimessi_guariti'] - df_r_0['dimessi_guariti']
df_r['np_su_nt'] = ((df_r['nuovi_positivi'] / df_r['nuovi_tamponi']) * 100).round(2)
df_r['np_su_p'] = ((df_r['nuovi_positivi'] / df_r['popolazione']) * 100).round(5)
df_r['tc_su_p'] = ((df_r['totale_casi'] / df_r['popolazione']) * 100).round(2)

df_r = df_r.sort_values(by='nuovi_positivi', ascending=False).reset_index(drop=True)
df_r

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,saturazione_terapia_intensiva,variazione_isolamento_domiciliare,variazione_nuovi_positivi,nuovi_tamponi,nuovi_casi_testati,nuovi_deceduti,nuovi_dimessi_guariti,np_su_nt,np_su_p,tc_su_p
0,2020-11-20T17:00:00,3,Lombardia,8304,930,9234,150407,159641,4061,9221,...,89.77,4033,1768,42248,13213.0,175,4985,21.83,0.09126,3.54
1,2020-11-20T17:00:00,15,Campania,2244,201,2445,96547,98992,2806,4226,...,39.80,2842,892,27649,16941.0,25,1395,15.28,0.07304,2.24
2,2020-11-20T17:00:00,1,Piemonte,5225,393,5618,70927,76545,-692,3861,...,68.35,-773,-1488,21513,9801.0,88,4465,17.95,0.08893,3.32
3,2020-11-20T17:00:00,5,Veneto,2170,267,2437,66250,68687,2105,3468,...,26.28,2068,-285,15966,5186.0,66,1297,21.72,0.07066,2.37
4,2020-11-20T17:00:00,12,Lazio,3202,337,3539,74207,77746,2205,2667,...,39.79,2098,-30,26887,21922.0,41,421,9.92,0.04547,1.66
5,2020-11-20T17:00:00,8,Emilia-Romagna,2534,244,2778,60156,62934,1925,2533,...,38.49,1880,373,22307,8449.0,47,558,11.36,0.05670,2.25
6,2020-11-20T17:00:00,9,Toscana,1792,295,2087,51782,53869,-255,2207,...,56.41,-241,235,18442,8340.0,48,2414,11.97,0.05928,2.44
7,2020-11-20T17:00:00,19,Sicilia,1537,242,1779,32977,34756,1175,1634,...,41.16,1168,-237,10020,6405.0,43,416,16.31,0.03289,1.01
8,2020-11-20T17:00:00,16,Puglia,1408,202,1610,27824,29434,934,1456,...,54.74,848,193,10102,4728.0,16,506,14.41,0.03632,1.02
9,2020-11-20T17:00:00,6,Friuli Venezia Giulia,505,54,559,11616,12175,530,1018,...,30.00,504,-179,7522,2086.0,28,460,13.53,0.08404,1.89


In [14]:
# Write pre-processed data
df_r.to_csv(csv_out_name, index=False)

In [15]:
# ICU Stats
df_r[['denominazione_regione', 'terapia_intensiva', 'posti_disponibili_terapia_intensiva', 'totale_posti_terapia_intensiva', 'saturazione_terapia_intensiva']] \
    .sort_values(by='saturazione_terapia_intensiva', ascending=False).reset_index(drop=True)

,denominazione_regione,terapia_intensiva,posti_disponibili_terapia_intensiva,totale_posti_terapia_intensiva,saturazione_terapia_intensiva
0,Lombardia,930,106,1036,89.77
1,Trentino-Alto Adige,76,30,106,71.70
2,Piemonte,393,182,575,68.35
3,Umbria,71,46,117,60.68
4,Liguria,120,89,209,57.42
5,Marche,81,62,143,56.64
6,Toscana,295,228,523,56.41
7,Puglia,202,167,369,54.74
8,Abruzzo,75,73,148,50.68
9,Valle d'Aosta,10,10,20,50.00


In [16]:
def add_cols(df):
    df['variazione_terapia_intensiva'] = df['terapia_intensiva'].diff()
    df['variazione_ricoverati_con_sintomi'] = df['ricoverati_con_sintomi'].diff()
    df['variazione_totale_ospedalizzati'] = df['totale_ospedalizzati'].diff()
    df['nuovi_dimessi_guariti'] = df['dimessi_guariti'].diff()
    df['nuovi_deceduti'] = df['deceduti'].diff()
    df['nuovi_tamponi'] = df['tamponi'].diff()
    df['nuovi_casi_testati'] = df['casi_testati'].diff()
    df['variazione_isolamento_domiciliare'] = df['isolamento_domiciliare'].diff()
    df = df.iloc[1:]
    df['ti_su_to'] = ((df['terapia_intensiva'] / df['totale_ospedalizzati']).replace(np.inf, 0) * 100).round(2)
    df['to_su_tp'] = ((df['totale_ospedalizzati'] / df['totale_positivi']).replace(np.inf, 0) * 100).round(2)
    df['np_su_nt'] = ((df['nuovi_positivi'] / df['nuovi_tamponi']).replace(np.inf, 0) * 100).round(2)
    df['np_su_nct'] = ((df['nuovi_positivi'] / df['nuovi_casi_testati']).replace(np.inf, 0) * 100).round(2)
    df['var_ti_perc'] = ((df['variazione_terapia_intensiva'] / (df['terapia_intensiva'] - df['variazione_terapia_intensiva'])).replace(np.inf, 0) * 100).round(2)
    df['var_pos_perc'] = ((df['nuovi_positivi'] / (df['totale_positivi'] - df['nuovi_positivi'] + df['nuovi_dimessi_guariti'] + df['nuovi_deceduti'])).replace(np.inf, 0) * 100).round(3)
    return df.fillna(0)

In [17]:
# df_rd.sort_values(by='codice_regione').reset_index(drop=True)
df_rn = pd.read_csv(regions_name)
for region_code in df_rn.codice_regione:
    if region_code != 4: # TAA
        df_cur_region = df_rd[df_rd.codice_regione == region_code]
        region_name = df_cur_region.denominazione_regione.iloc[0].lower()
        csv_out_region_name = '{path_csv}/regioni/covid19-{region_name}.csv'.format(path_csv=path_csv, region_name=region_name)
        df_cur_region = add_cols(df_cur_region)
        df_cur_region.to_csv(csv_out_region_name, index=False)
        print('Aggiornato {file}'.format(file=csv_out_region_name))

Aggiornato ../data/csv/regioni/covid19-piemonte.csv
Aggiornato ../data/csv/regioni/covid19-valle d'aosta.csv
Aggiornato ../data/csv/regioni/covid19-lombardia.csv
Aggiornato ../data/csv/regioni/covid19-veneto.csv
Aggiornato ../data/csv/regioni/covid19-friuli venezia giulia.csv
Aggiornato ../data/csv/regioni/covid19-liguria.csv
Aggiornato ../data/csv/regioni/covid19-emilia-romagna.csv
Aggiornato ../data/csv/regioni/covid19-toscana.csv
Aggiornato ../data/csv/regioni/covid19-umbria.csv
Aggiornato ../data/csv/regioni/covid19-marche.csv
Aggiornato ../data/csv/regioni/covid19-lazio.csv
Aggiornato ../data/csv/regioni/covid19-abruzzo.csv
Aggiornato ../data/csv/regioni/covid19-molise.csv
Aggiornato ../data/csv/regioni/covid19-campania.csv
Aggiornato ../data/csv/regioni/covid19-puglia.csv
Aggiornato ../data/csv/regioni/covid19-basilicata.csv
Aggiornato ../data/csv/regioni/covid19-calabria.csv
Aggiornato ../data/csv/regioni/covid19-sicilia.csv
Aggiornato ../data/csv/regioni/covid19-sardegna.csv


In [18]:
df_tb = df_rd[df_rd.codice_regione > 20].reset_index(drop=True)
df_tb

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati
0,2020-02-24T18:00:00,21,P.A. Bolzano,0,0,0,0,0,0,0,0,0,NaN,NaN,0,1,NaN
1,2020-02-24T18:00:00,22,P.A. Trento,0,0,0,0,0,0,0,0,0,NaN,NaN,0,3,NaN
2,2020-02-25T18:00:00,21,P.A. Bolzano,1,0,1,0,1,1,1,0,0,NaN,NaN,1,1,NaN
3,2020-02-25T18:00:00,22,P.A. Trento,0,0,0,0,0,0,0,0,0,NaN,NaN,0,3,NaN
4,2020-02-26T18:00:00,21,P.A. Bolzano,1,0,1,0,1,0,0,0,0,NaN,NaN,1,1,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
537,2020-11-18T17:00:00,22,P.A. Trento,398,38,436,2424,2860,-134,236,9852,549,6982.0,6279.0,13261,337891,124544.0
538,2020-11-19T17:00:00,21,P.A. Bolzano,469,39,508,10696,11204,303,696,7739,438,19381.0,0.0,19381,285421,141552.0
539,2020-11-19T17:00:00,22,P.A. Trento,407,39,446,2540,2986,126,266,9985,556,7096.0,6431.0,13527,340985,125144.0
540,2020-11-20T17:00:00,21,P.A. Bolzano,468,38,506,11005,11511,307,736,8160,446,20117.0,0.0,20117,288707,142658.0


In [19]:
# Initialize df_taa
df_taa = Functions.replace_bt_with_taa(df_tb[:2])
for i in range(2, df_tb.shape[0] - 2, 2):
    df_taa = df_taa.append(Functions.replace_bt_with_taa(df_tb[i:i+2]), ignore_index=True)
df_taa = add_cols(df_taa)
df_taa

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,nuovi_deceduti,nuovi_tamponi,nuovi_casi_testati,variazione_isolamento_domiciliare,ti_su_to,to_su_tp,np_su_nt,np_su_nct,var_ti_perc,var_pos_perc
1,2020-02-25T18:00:00,4,Trentino-Alto Adige,1,0,1,0,1,1,1,...,0.0,0.0,0.0,0.0,0.00,100.00,0.00,0.00,0.00,0.000
2,2020-02-26T18:00:00,4,Trentino-Alto Adige,1,0,1,0,1,0,0,...,0.0,0.0,0.0,0.0,0.00,100.00,0.00,0.00,0.00,0.000
3,2020-02-27T18:00:00,4,Trentino-Alto Adige,1,0,1,0,1,0,0,...,0.0,30.0,0.0,0.0,0.00,100.00,0.00,0.00,0.00,0.000
4,2020-02-28T18:00:00,4,Trentino-Alto Adige,1,0,1,0,1,0,0,...,0.0,0.0,0.0,0.0,0.00,100.00,0.00,0.00,0.00,0.000
5,2020-02-29T17:00:00,4,Trentino-Alto Adige,1,0,1,0,1,0,0,...,0.0,25.0,0.0,0.0,0.00,100.00,0.00,0.00,0.00,0.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
265,2020-11-15T17:00:00,4,Trentino-Alto Adige,849,76,925,12327,13252,359,896,...,27.0,5408.0,1516.0,320.0,8.22,6.98,16.57,59.10,1.33,6.950
266,2020-11-16T17:00:00,4,Trentino-Alto Adige,873,75,948,12458,13406,154,725,...,22.0,4008.0,1669.0,131.0,7.91,7.07,18.09,43.44,-1.32,5.471
267,2020-11-17T17:00:00,4,Trentino-Alto Adige,870,79,949,12746,13695,289,534,...,12.0,2958.0,1057.0,288.0,8.32,6.93,18.05,50.52,5.33,3.983
268,2020-11-18T17:00:00,4,Trentino-Alto Adige,879,77,956,12805,13761,66,817,...,18.0,7692.0,1965.0,59.0,8.05,6.95,10.62,41.58,-2.53,5.966


In [20]:
region_name = df_taa.denominazione_regione.iloc[0].lower()
csv_out_region_name = '{path_csv}/regioni/covid19-{region_name}.csv'.format(path_csv=path_csv, region_name=region_name)
df_taa.to_csv(csv_out_region_name, index=False)
print('Aggiornato {file}'.format(file=csv_out_region_name))

Aggiornato ../data/csv/regioni/covid19-trentino-alto adige.csv
